In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# https://huggingface.co/Helsinki-NLP/opus-mt-ko-en

model_checkpoint = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [3]:
import torch
torch.__version__

'2.0.0+cu117'

In [4]:
# dataset
from datasets import load_dataset

dataset = load_dataset('json', data_files='./일상생활및구어체_영한_train_set.json', field='data')

Found cached dataset json (/home/juhwan/.cache/huggingface/datasets/json/default-23e8ad3d1a915c98/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 1/1 [00:00<00:00, 13.25it/s]


In [5]:
split_datasets = dataset["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

Loading cached split indices for dataset at /home/juhwan/.cache/huggingface/datasets/json/default-23e8ad3d1a915c98/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-cfd229826466e59a.arrow and /home/juhwan/.cache/huggingface/datasets/json/default-23e8ad3d1a915c98/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-61c3fa9b299534e5.arrow


In [6]:
# declare preprocessing code 

max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = examples['ko']
    targets = examples['en']

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs            

In [7]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Loading cached processed dataset at /home/juhwan/.cache/huggingface/datasets/json/default-23e8ad3d1a915c98/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-88111a08c390f2b4.arrow
Loading cached processed dataset at /home/juhwan/.cache/huggingface/datasets/json/default-23e8ad3d1a915c98/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e/cache-3ac33a9c4b7e49d2.arrow


In [8]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [9]:
from datasets import load_metric
metric = load_metric("sacrebleu")

/tmp/ipykernel_1263/3063495125.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [10]:
# 코드 자세히봐야해.. 왜 -100을 제거하고 그런거 알아봐야한다.
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우.
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100은 건너뛴다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [11]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"finetuned-aihub-ko-to-en",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [12]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [13]:
# 학습하기전 초기모델 점수
trainer.evaluate(max_length=max_target_length)

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.6437932252883911,
 'eval_bleu': 23.463732945313037,
 'eval_runtime': 4407.0119,
 'eval_samples_per_second': 27.236,
 'eval_steps_per_second': 0.106}

In [14]:
trainer.train()

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


TrainOutput(global_step=25320, training_loss=1.1216386442500834, metrics={'train_runtime': 7640.2382, 'train_samples_per_second': 424.179, 'train_steps_per_second': 3.314, 'total_flos': 2.7898742048292864e+16, 'train_loss': 1.1216386442500834, 'epoch': 3.0})

In [15]:
trainer.evaluate(max_length=max_target_length)

{'eval_loss': 1.0371508598327637,
 'eval_bleu': 36.032725201690134,
 'eval_runtime': 2768.7492,
 'eval_samples_per_second': 43.352,
 'eval_steps_per_second': 0.169,
 'epoch': 3.0}

In [16]:
from transformers import pipeline
translator = pipeline('translation', model=model_checkpoint)

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [17]:
translator('나는 불고기가 먹고 싶어.')

[{'translation_text': 'I want beef.'}]